In [1]:
import numpy as np
import random
import time

### Genetic Algorithem

In [5]:
# Calculate total distance of a route
def calculate_distance(route, matrix):
    total = 0
    for i in range(len(route) - 1):
        total += matrix[route[i]][route[i + 1]]
    return total

def random_route(n):
    route = []
    while len(route) < n:
        city = random.randint(0, n - 1)
        if city not in route:
            route.append(city)
    return route

def mutate(route):
    # swap two cities
    i = random.randint(0, len(route) - 1)
    j = random.randint(0, len(route) - 1)
    route[i], route[j] = route[j], route[i]
    return route

def crossover(p1, p2):
    # simple ordered crossover
    start = random.randint(0, len(p1) - 2)
    end = random.randint(start + 1, len(p1) - 1)
    child = [-1] * len(p1)

    for i in range(start, end):
        child[i] = p1[i]

    fill_pos = 0
    for city in p2:
        if city not in child:
            while child[fill_pos] != -1:
                fill_pos += 1
            child[fill_pos] = city

    return child

def ga(matrix, num_cities):
    population_size = 100
    generations = 300
    mutation_rate = 0.1

    population = []
    for _ in range(population_size):
        route = random_route(num_cities)
        population.append(route)

    best = population[0]
    best_dist = calculate_distance(best + [best[0]], matrix)

    for gen in range(generations):
        # sort by distance
        population.sort(key=lambda r: calculate_distance(r + [r[0]], matrix))
        new_population = population[:10]  # elitism

        while len(new_population) < population_size:
            parent1 = population[random.randint(0, 49)]
            parent2 = population[random.randint(0, 49)]
            child = crossover(parent1, parent2)
            #some times we apply mutation to child
            if random.random() < mutation_rate:
                child = mutate(child)
            new_population.append(child)

        population = new_population
        best_candidate = population[0]
        best_candidate_dist = calculate_distance(best_candidate + [best_candidate[0]], matrix)
        if best_candidate_dist < best_dist:
            best = best_candidate
            best_dist = best_candidate_dist

    return best + [best[0]], best_dist

### Ant Colony Tsp

In [6]:
def aco(matrix, num_cities):
    # مقداردهی اولیه ماتریس فرومون‌ها)
    pheromone = np.ones((num_cities, num_cities))

    # مسیرهای کوتاه‌تر "دید" بیشتری دارند (جذاب‌ترند)
    visibility = [[0 if i == j else 1 / matrix[i][j] for j in range(num_cities)] for i in range(num_cities)]

    alpha = 1         # p
    beta = 5          # (v)
    evaporation = 0.5 
    Q = 100           # ضریب تقویت فرومون
    iterations = 100 
    num_ants = 20    

    best_route = None           
    best_length = float('inf')  

    for _ in range(iterations):
        routes = []  
        lengths = [] 

        for _ in range(num_ants):
            visited = []  
            current = random.randint(0, num_cities - 1)  # starting with random city
            visited.append(current)

            while len(visited) < num_cities:
                probs = []  
                total = 0  

                for next_city in range(num_cities):
                    if next_city not in visited:
                        # محاسبه احتمال حرکت به شهر بعدی
                        ph = pheromone[current][next_city] ** alpha
                        vis = visibility[current][next_city] ** beta
                        prob = ph * vis
                        probs.append((next_city, prob))
                        total += prob

                if total == 0:
                    break  
                # slecting next city
                r = random.random()
                cumulative = 0
                for city, prob in probs:
                    cumulative += prob / total
                    if r <= cumulative:
                        next_city = city
                        break

                visited.append(next_city)
                current = next_city

            visited.append(visited[0])  # adding first city for loop
            length = calculate_distance(visited, matrix)  

            routes.append(visited)
            lengths.append(length)

            if length < best_length:
                best_length = length
                best_route = visited


        #pheromone update 

        pheromone *= (1 - evaporation)

        for i in range(len(routes)):
            path = routes[i]
            length = lengths[i]
            for j in range(num_cities):
                a = path[j]
                b = path[j + 1]
                pheromone[a][b] += Q / length 

    return best_route, best_length 


### Checking Algorithems

In [20]:
with open("in.txt", 'r') as f:
    lines = [line.strip() for line in f if line.strip()]

problem = 0
i = 0
while i < len(lines):
    num_cities = int(lines[i])
    i += 1
    matrix = []
    for _ in range(num_cities):
        row = list(map(int, lines[i].split()))
        matrix.append([float('inf') if x == -1 else x for x in row])
        i += 1
    problem +=1    
    print(f"\n===== problem number : {(problem)} =====")
    # Genetic Algorithm
    start = time.time()
    ga_route, ga_length = ga(matrix, num_cities)
    ga_time = time.time() - start
    # Ant Colony Optimization
    start = time.time()
    aco_route, aco_length = aco(matrix, num_cities)
    aco_time = time.time() - start
    # چاپ نتایج
    print("GA path:", ga_route)
    print("GA length:", ga_length)
    print("GA time:", round(ga_time, 4), "seconds")
    print("ACO path:",  aco_route)
    print("ACO length:", aco_length)
    print("ACO time:",  round(aco_time, 4), "seconds")


===== problem number : 1 =====
GA path: [2, 3, 0, 1, 2]
GA length: 4
GA time: 0.0826 seconds
ACO path: [0, 1, 2, 3, 0]
ACO length: 4
ACO time: 0.0152 seconds

===== problem number : 2 =====
GA path: [5, 9, 6, 8, 3, 4, 1, 2, 7, 0, 5]
GA length: 48
GA time: 0.1085 seconds
ACO path: [8, 3, 4, 1, 2, 7, 0, 5, 9, 6, 8]
ACO length: 48
ACO time: 0.0719 seconds

===== problem number : 3 =====
GA path: [3, 2, 5, 4, 0, 1, 6, 3]
GA length: 18
GA time: 0.0812 seconds
ACO path: [1, 6, 3, 0, 4, 5, 2, 1]
ACO length: 18
ACO time: 0.0363 seconds

===== problem number : 4 =====
GA path: [2, 0, 1, 3, 4, 2]
GA length: 26
GA time: 0.073 seconds
ACO path: [1, 0, 4, 2, 3, 1]
ACO length: 26
ACO time: 0.0194 seconds
